In [1]:
import cv2
import json
import os
from _paths import nomeroff_net_dir
from nomeroff_net.pipes.number_plate_text_readers.text_detector import TextDetector

In [8]:
def compare(model_path="latest",
            text_detector_name = "eu",
            img_format = "png",
            root_dir=os.path.join(nomeroff_net_dir,  './data/dataset/TextDetector/ocr_example/val'),
            predicted_part_size=1000,
            replace_tamplate = None):
    if replace_tamplate is None:
        replace_tamplate = {'moderation': {'isModerated': 1, 'moderatedBy': 'ApelSYN'}}
    text_detector = TextDetector({
        text_detector_name: {
            "for_regions": [""],
            "model_path": model_path
        },
    })

    ann_dir = os.path.join(root_dir, "ann")
    jsons = []
    jsons_paths = []
    for dir_name, subdir_list, file_list in os.walk(ann_dir):
        for fname in file_list:
            fname = os.path.join(ann_dir, fname)
            jsons_paths.append(fname)
            with open(fname) as jsonF:
                jsonData = json.load(jsonF)
            jsons.append(jsonData)
    print("LOADED {} ANNOTATIONS".format(len(jsons)))

    img_dir = os.path.join(root_dir, "img")
    imgs = []                
    for j in jsons:
        img_path =os.path.join(img_dir, "{}.{}".format(j["name"], img_format))
        img = cv2.imread(img_path)
        imgs.append(img)
    print("LOADED {} IMAGES".format(len(imgs)))

    predicted = []
    N = int(len(imgs) / predicted_part_size) + 1
    for i in range(N):
        part = i*predicted_part_size
        part_imgs = imgs[part:part+predicted_part_size]
        model_inputs = text_detector.preprocess(part_imgs, ["" for _ in part_imgs], [1 for _ in part_imgs])
        model_outputs = text_detector.forward(model_inputs)
        predicted_part = text_detector.postprocess(model_outputs)
        predicted += predicted_part

    print("PREDICTED {} IMAGES".format(len(predicted)))

    err_cnt = 0
    for i in range(len(jsons_paths)):
        json_path      = jsons_paths[i]
        predicted_item = predicted[i]
        jsonData       = jsons[i]
        jsonData["moderation"]["predicted"] = predicted_item
        if jsonData["description"] == jsonData["moderation"]["predicted"]:
            jsonData.update(replace_tamplate)
            jsonData["moderation"]["isModerated"] = 1
        else:
            print("Predicted '{}', real: '{}' in file {}".format(jsonData["moderation"]["predicted"],jsonData["description"], json_path))
            err_cnt = err_cnt+1

        with open(json_path, "w") as jsonWF:
            json.dump(jsonData, jsonWF)

    print("Error detection count: {}".format(err_cnt))
    print("Accuracy: {}".format(1-err_cnt/len(predicted)))

In [9]:
compare()

LOADED 1 ANNOTATIONS
LOADED 1 IMAGES
PREDICTED 1 IMAGES
Predicted 'Y929', real: '0038SC' in file /mnt/data/var/www/nomeroff-net/tools/py/../.././data/dataset/TextDetector/ocr_example/val/ann/0038SC-0.json
Error detection count: 1
Accuracy: 0.0
